In [1]:
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import functions as func
from pyspark.sql.types import StringType,FloatType
import nltk
from pyspark.sql import SparkSession


In [2]:
# reading in flight data csv
spark = SparkSession.builder.master('local[*]').appName('FlightAnalysis').getOrCreate()
combinedFlightsDF = spark.read.options(header = True, inferSchema = True).csv("Combined_Flights_2018.csv")

22/11/25 16:08:07 WARN Utils: Your hostname, Jeanas-MacBook-Pro-5.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
22/11/25 16:08:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/25 16:08:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# drops unnecessary columns
drop_cols = ("FlightDate","Origin","Dest","Diverted","DepDelayMinutes","ArrDelayMinutes","IATA_Code_Operating_Airline","DestAirportSeqID","DestCityMarketID","DOT_ID_Operating_Airline","DOT_ID_Marketing_Airline","IATA_Code_Marketing_Airline","Operated_or_Branded_Code_Share_Partners","CRSDepTime","DepDelay","AirTime","CRSElapsedTime", "Distance", "Marketing_Airline_Network", "Flight_Number_Marketing_Airline", "Operating_Airline", "Tail_Number", "Flight_Number_Operating_Airline", "OriginAirportID",  "OriginAirportSeqID", "OriginCityMarketID",  "OriginState",  "OriginStateFips", "OriginWac", "DestAirportID", "DestState", "DestStateFips", "DestStateName", "DestWac", "DepartureDelayGroups", "DepTimeBlk", "TaxiOut", "WheelsOff", "WheelsOn", "TaxiIn", "CRSArrTime", "ArrDelay", "ArrivalDelayGroups", "ArrTimeBlk", "DistanceGroup", "DivAirportLandings")
combinedFlightsDF = combinedFlightsDF.drop(*drop_cols)

In [70]:
combinedFlightsDF.show(5)

+---------+---------+-------+-------+-----------------+----+-------+-----+----------+---------+---------------+---------------+------------+--------+--------+
|  Airline|Cancelled|DepTime|ArrTime|ActualElapsedTime|Year|Quarter|Month|DayofMonth|DayOfWeek| OriginCityName|OriginStateName|DestCityName|DepDel15|ArrDel15|
+---------+---------+-------+-------+-----------------+----+-------+-----+----------+---------+---------------+---------------+------------+--------+--------+
|Envoy Air|    false| 1209.0| 1350.0|            101.0|2019|      2|    4|         1|        1|Little Rock, AR|       Arkansas| Chicago, IL|     0.0|     0.0|
|Envoy Air|    false| 1200.0| 1348.0|            108.0|2019|      2|    4|         2|        2|Little Rock, AR|       Arkansas| Chicago, IL|     0.0|     0.0|
|Envoy Air|    false| 1203.0| 1342.0|             99.0|2019|      2|    4|         3|        3|Little Rock, AR|       Arkansas| Chicago, IL|     0.0|     0.0|
|Envoy Air|    false| 1435.0| 1621.0|         

In [4]:
searchBarDropCols = ("Cancelled","Quarter","DayOfWeek")
searchBarDF = combinedFlightsDF.drop(*searchBarDropCols)

In [6]:
searchBarDF.show(5)

+---------+-------+-------+-----------------+----+-----+----------+---------------+------------+--------+--------+
|  Airline|DepTime|ArrTime|ActualElapsedTime|Year|Month|DayofMonth| OriginCityName|DestCityName|DepDel15|ArrDel15|
+---------+-------+-------+-----------------+----+-----+----------+---------------+------------+--------+--------+
|Envoy Air| 1209.0| 1350.0|            101.0|2019|    4|         1|Little Rock, AR| Chicago, IL|     0.0|     0.0|
|Envoy Air| 1200.0| 1348.0|            108.0|2019|    4|         2|Little Rock, AR| Chicago, IL|     0.0|     0.0|
|Envoy Air| 1203.0| 1342.0|             99.0|2019|    4|         3|Little Rock, AR| Chicago, IL|     0.0|     0.0|
|Envoy Air| 1435.0| 1621.0|            106.0|2019|    4|         4|Little Rock, AR| Chicago, IL|     1.0|     1.0|
|Envoy Air| 1216.0| 1410.0|            114.0|2019|    4|         5|Little Rock, AR| Chicago, IL|     0.0|     0.0|
+---------+-------+-------+-----------------+----+-----+----------+-------------

In [155]:
#Creates csv with specified data (DONT RUN UNLESS NEEDED)
searchBarDF.write.option("header",True).csv("allFlightsSearchBar")

In [120]:
#dont need to run
# Find # of flights per year
yearCount = combinedFlightsDF.groupBy("Year").count()
yearCount.show()

+----+-------+
|Year|  count|
+----+-------+
|2020|5022397|
+----+-------+



In [5]:
# creating new df to analyze delayed flights by airline, orgin city, and month/year
drop_cols = ("DestCityName", "DayOfMonth", "DepTime", "ArrTime", "ActualElapsedTime")
delayDF = combinedFlightsDF.drop(*drop_cols)
delayDF.show(5)

+-----------------+---------+----+-------+-----+---------+--------------+---------------+--------+--------+
|          Airline|Cancelled|Year|Quarter|Month|DayOfWeek|OriginCityName|OriginStateName|DepDel15|ArrDel15|
+-----------------+---------+----+-------+-----+---------+--------------+---------------+--------+--------+
|Endeavor Air Inc.|    false|2018|      1|    1|        2|    Albany, GA|        Georgia|     0.0|     0.0|
|Endeavor Air Inc.|    false|2018|      1|    1|        3|    Albany, GA|        Georgia|     0.0|     0.0|
|Endeavor Air Inc.|    false|2018|      1|    1|        4|    Albany, GA|        Georgia|     0.0|     0.0|
|Endeavor Air Inc.|    false|2018|      1|    1|        5|    Albany, GA|        Georgia|     0.0|     0.0|
|Endeavor Air Inc.|    false|2018|      1|    1|        6|    Albany, GA|        Georgia|     0.0|     0.0|
+-----------------+---------+----+-------+-----+---------+--------------+---------------+--------+--------+
only showing top 5 rows



In [ ]:
delayDF.columns

In [6]:
# uses bitwise or operator to find overall delayed flights
delayDF = delayDF.withColumn("ifDelayed", func.when((delayDF.ArrDel15 > 0) | (delayDF.DepDel15 > 0), 1).otherwise(0))
delayDF.show(10)

+-----------------+---------+----+-------+-----+---------+--------------+---------------+--------+--------+---------+
|          Airline|Cancelled|Year|Quarter|Month|DayOfWeek|OriginCityName|OriginStateName|DepDel15|ArrDel15|ifDelayed|
+-----------------+---------+----+-------+-----+---------+--------------+---------------+--------+--------+---------+
|Endeavor Air Inc.|    false|2018|      1|    1|        2|    Albany, GA|        Georgia|     0.0|     0.0|        0|
|Endeavor Air Inc.|    false|2018|      1|    1|        3|    Albany, GA|        Georgia|     0.0|     0.0|        0|
|Endeavor Air Inc.|    false|2018|      1|    1|        4|    Albany, GA|        Georgia|     0.0|     0.0|        0|
|Endeavor Air Inc.|    false|2018|      1|    1|        5|    Albany, GA|        Georgia|     0.0|     0.0|        0|
|Endeavor Air Inc.|    false|2018|      1|    1|        6|    Albany, GA|        Georgia|     0.0|     0.0|        0|
|Endeavor Air Inc.|    false|2018|      1|    1|        

In [7]:
#create column making cancelled into bool (integer) value
delayDF = delayDF.withColumn('ifCancelled', func.when(delayDF.Cancelled == 'false', 0).otherwise(1))
delayDF.show(10)

+-----------------+---------+----+-------+-----+---------+--------------+---------------+--------+--------+---------+-----------+
|          Airline|Cancelled|Year|Quarter|Month|DayOfWeek|OriginCityName|OriginStateName|DepDel15|ArrDel15|ifDelayed|ifCancelled|
+-----------------+---------+----+-------+-----+---------+--------------+---------------+--------+--------+---------+-----------+
|Endeavor Air Inc.|    false|2018|      1|    1|        2|    Albany, GA|        Georgia|     0.0|     0.0|        0|          0|
|Endeavor Air Inc.|    false|2018|      1|    1|        3|    Albany, GA|        Georgia|     0.0|     0.0|        0|          0|
|Endeavor Air Inc.|    false|2018|      1|    1|        4|    Albany, GA|        Georgia|     0.0|     0.0|        0|          0|
|Endeavor Air Inc.|    false|2018|      1|    1|        5|    Albany, GA|        Georgia|     0.0|     0.0|        0|          0|
|Endeavor Air Inc.|    false|2018|      1|    1|        6|    Albany, GA|        Georgia| 

In [159]:
# groups delayed flights by airline
numDelayedByAirline = delayDF.groupBy("Airline").agg({"ifDelayed" : "sum"})
numDelayedByAirline = numDelayedByAirline.withColumnRenamed("sum(ifDelayed)", "2018SumDelayedByAirline") #CHANGED
numDelayedByAirline.show(30,truncate=False)

# to check if delayed flight per airline add up to original amount
# sumdf = numDelayed.agg({"sum(ifDelayed)" : "sum"})
# sumdf.show()

+-----------------------------------------+----------------------+
|Airline                                  |allSumDelayedByAirline|
+-----------------------------------------+----------------------+
|Endeavor Air Inc.                        |147358                |
|United Air Lines Inc.                    |510700                |
|Compass Airlines                         |31705                 |
|Comair Inc.                              |188171                |
|Southwest Airlines Co.                   |1269581               |
|ExpressJet Airlines Inc.                 |79802                 |
|JetBlue Airways                          |325465                |
|Empire Airlines Inc.                     |3665                  |
|Envoy Air                                |205502                |
|Capital Cargo International              |68581                 |
|Hawaiian Airlines Inc.                   |39344                 |
|Mesa Airlines Inc.                       |155645             

In [160]:
#total flights for each of the 28 airlines
airlineRank = combinedFlightsDF.groupBy("Airline").count() #28 airlines
airlineRank = airlineRank.withColumnRenamed("count", "allflightCountByAirline") #CHANGED

In [89]:
airlineRank.show()

+--------------------+------------------------+
|             Airline|2021flightCountByAirline|
+--------------------+------------------------+
|GoJet Airlines, L...|                   60143|
|   Endeavor Air Inc.|                  266867|
|       Allegiant Air|                  115881|
|SkyWest Airlines ...|                  753343|
|         Horizon Air|                  111628|
|United Air Lines ...|                  446837|
|Air Wisconsin Air...|                   80256|
|         Comair Inc.|                  222602|
|Frontier Airlines...|                  137142|
|Southwest Airline...|                 1064640|
|Commutair Aka Cha...|                   76666|
|           Envoy Air|                  255751|
|Hawaiian Airlines...|                   60542|
|Delta Air Lines Inc.|                  747998|
|  Mesa Airlines Inc.|                  159789|
|American Airlines...|                  736399|
|   Republic Airlines|                  332926|
|    Spirit Air Lines|                  

In [161]:
#1 add a new column to show ratio of delayed flights for each airline
numDelayedByAirline = numDelayedByAirline\
    .join(airlineRank, "Airline")\
    .withColumn("alldelayedRatio", ((func.col("allSumDelayedByAirline") / func.col("allflightCountByAirline")) * 100))\
    .drop("count")
#CHANGE

In [91]:
#1
numDelayedByAirline.show(10)

+--------------------+-----------------------+------------------------+------------------+
|             Airline|2021sumDelayedByAirline|2021flightCountByAirline|  2021delayedRatio|
+--------------------+-----------------------+------------------------+------------------+
|GoJet Airlines, L...|                  12563|                   60143|20.888548958316015|
|   Endeavor Air Inc.|                  27145|                  266867| 10.17173348521923|
|       Allegiant Air|                  35903|                  115881|30.982645990283135|
|SkyWest Airlines ...|                 135451|                  753343| 17.97999052224551|
|         Horizon Air|                  16434|                  111628|14.722112731572723|
|United Air Lines ...|                  94483|                  446837| 21.14484700237447|
|Air Wisconsin Air...|                  16547|                   80256| 20.61777312599681|
|         Comair Inc.|                  37333|                  222602|16.771188039640254|

In [162]:
#Creates csv with specified data (DONT RUN UNLESS NEEDED)
numDelayedByAirline.write.option("header",True).csv("allnumDelayedByAirline")

In [163]:
#total flights for each of the origin cities
cityRank = combinedFlightsDF.groupBy("OriginCityName").count() 
cityRank = cityRank.withColumnRenamed("count", "alltotalFlightsByCity") #CHANGE

In [164]:
# groups delayed flights by city
numDelayedByCity = delayDF.groupBy("OriginCityName").agg({"ifDelayed" : "sum"})
numDelayedByCity = numDelayedByCity.withColumnRenamed("sum(ifDelayed)", "allsumDelayedByCity") #CHANGE
numDelayedByCity.show(30,truncate=False)

+------------------------+-------------------+
|OriginCityName          |allsumDelayedByCity|
+------------------------+-------------------+
|Gainesville, FL         |2847               |
|Richmond, VA            |19749              |
|Ontario, CA             |15887              |
|Pago Pago, TT           |113                |
|Tucson, AZ              |11284              |
|Myrtle Beach, SC        |10761              |
|Medford, OR             |5562               |
|Palm Springs, CA        |9311               |
|Durango, CO             |3502               |
|Corpus Christi, TX      |3734               |
|Mobile, AL              |4294               |
|Dubuque, IA             |442                |
|Pensacola, FL           |8954               |
|Huntsville, AL          |6572               |
|Fort Myers, FL          |31883              |
|Columbus, GA            |894                |
|Springfield, IL         |1419               |
|San Juan, PR            |27812              |
|Montrose/Del

In [165]:
#2 add a new column to show ratio of delayed flights for each airline
numDelayedByCity = numDelayedByCity\
    .join(cityRank, "OriginCityName")\
    .withColumn("alldelayedRatio", ((func.col("allSumDelayedByCity") / func.col("alltotalFlightsByCity")) * 100))\
    .drop("count")
#CHANGE

In [166]:
#2
numDelayedByCity.show(5)

+---------------+-------------------+---------------------+------------------+
| OriginCityName|allsumDelayedByCity|alltotalFlightsByCity|   alldelayedRatio|
+---------------+-------------------+---------------------+------------------+
|Gainesville, FL|               2847|                16524| 17.22948438634713|
|   Richmond, VA|              19749|                96878|20.385433225293667|
|    Ontario, CA|              15887|                91776|17.310625871687588|
|  Pago Pago, TT|                113|                  299| 37.79264214046823|
|     Tucson, AZ|              11284|                70842|15.928404054092205|
+---------------+-------------------+---------------------+------------------+
only showing top 5 rows



In [167]:
#Creates csv with specified data (DONT RUN UNLESS NEEDED)
numDelayedByCity.write.option("header",True).csv("allnumDelayedByCity") #CHANGE

In [168]:
#Count total # of flights by Quarter and Origin State
timeLocationCancelledDF = delayDF.groupBy("Quarter","OriginStateName").count()
timeLocationCancelledDF = timeLocationCancelledDF.withColumnRenamed("count", "alltotalFlights") #CHANGE

In [ ]:
timeLocationCancelledDF.show()

In [169]:
#Count cancelled flight based on quarter and origin state name
timeLocationFlightsDF = delayDF.groupBy("Quarter","OriginStateName").agg({"ifCancelled" : "sum"})
timeLocationFlightsDF = timeLocationFlightsDF.withColumnRenamed("sum(ifCancelled)", "allcancelledFlights") #CHANGE

In [ ]:
timeLocationFlightsDF.show()

In [171]:
#join previous dataframes
timeLocationDF = timeLocationFlightsDF.join(timeLocationCancelledDF, ["Quarter", "OriginStateName"])

In [172]:
#calculate ration of cancelled flights / total flights based on quarter + origin state
timeLocationDF = timeLocationDF.withColumn("alltimeLocationCancellationRatio", ((func.col("allcancelledFlights") / func.col("alltotalFlights")) * 100))
#CHANGE

In [62]:
# Cancelation based on Time + Location
timeLocationDF.show()

+-------+-------------------+--------------------+----------------+---------------------------------+
|Quarter|    OriginStateName|2022cancelledFlights|2022totalFlights|2022timeLocationCancellationRatio|
+-------+-------------------+--------------------+----------------+---------------------------------+
|      2|               Ohio|                 742|           24548|                3.022649503014502|
|      2|      Massachusetts|                1277|           36848|               3.4655883630047764|
|      2|         New Mexico|                  65|            6278|               1.0353615801210576|
|      2|             Hawaii|                 233|           30792|                0.756690049363471|
|      2|            Vermont|                 114|            2535|                4.497041420118343|
|      2|             Nevada|                 763|           47811|               1.5958670598816171|
|      2|          Louisiana|                 358|           17803|               

In [173]:
#3 Creates csv with specified data (DONT RUN UNLESS NEEDED)
timeLocationDF.write.option("header",True).csv("alltimeLocationCancellation") #CHANGE

In [174]:
#4 delayed flights
delayedFlights = delayDF.groupBy("ifDelayed").count()
delayedFlights = delayedFlights.withColumnRenamed("count", "allcount") #CHANGE

In [ ]:
delayedFlights.show() #1 - Delayed, 0 - Not delayed

In [175]:
#Creates csv with specified data (DONT RUN UNLESS NEEDED)
delayedFlights.write.option("header",True).csv("alldelayedFlights") #CHANGE

In [176]:
#5 cancelled flights
cancelledFlights = combinedFlightsDF.groupBy("Cancelled").count()
cancelledFlights = cancelledFlights.withColumnRenamed("count", "allcount") #CHANGE

In [ ]:
cancelledFlights.show()

In [177]:
#Creates csv with specified data (DONT RUN UNLESS NEEDED)
cancelledFlights.write.option("header",True).csv("allcancelledFlights") #CHANGE

In [178]:
#categorize flights by date
dayOfWeekRank = combinedFlightsDF.groupBy("DayOfWeek").count()
dayOfWeekRank = dayOfWeekRank.withColumnRenamed("count", "alltotalFlightsOnDay") #CHANGE

In [70]:
dayOfWeekRank.show()

+---------+---------------------+
|DayOfWeek|2022totalFlightsOnDay|
+---------+---------------------+
|        1|               600370|
|        6|               535618|
|        3|               566951|
|        5|               605385|
|        4|               603939|
|        7|               609250|
|        2|               556805|
+---------+---------------------+



In [179]:
# group delayed flights by day of the week
delayedFlightByDay = delayDF.groupBy("DayOfWeek").agg({"ifDelayed" : "sum"})
delayedFlightByDay = delayedFlightByDay.withColumnRenamed("sum(ifDelayed)", "allsumDelayedByWeekDay") #TODO NO RENAMING

In [ ]:
delayedFlightByDay.show()

In [180]:
# group cancelled flights by day of the week
cancelledFlightByDay = delayDF.groupBy("DayOfWeek").agg({"ifCancelled" : "sum"})
cancelledFlightByDay = cancelledFlightByDay.withColumnRenamed("sum(ifCancelled)", "allsumCancelledByWeekDay") 

In [ ]:
cancelledFlightByDay.show()

In [181]:
analysisByWeekDay = dayOfWeekRank.join(delayedFlightByDay, ["DayOfWeek"]).join(cancelledFlightByDay, ["DayOfWeek"])

In [183]:
analysisByWeekDay = analysisByWeekDay.withColumn("allCancelledRatio", (( func.col("allsumCancelledByWeekDay") / func.col("alltotalFlightsOnDay")) * 100))

In [184]:
analysisByWeekDay = analysisByWeekDay.withColumn("allDelayedRatio", ((func.col("allsumDelayedByWeekDay") / func.col("alltotalFlightsOnDay")) * 100))

In [186]:
analysisByWeekDay = analysisByWeekDay.withColumn("allDelayed+CancelledRatio", (((func.col("allsumDelayedByWeekDay") + func.col("allsumCancelledByWeekDay")) / func.col("alltotalFlightsOnDay")) * 100))

In [187]:
#6 Days of the week analysis
analysisByWeekDay.show()

+---------+--------------------+----------------------+------------------------+------------------+------------------+-------------------------+
|DayOfWeek|alltotalFlightsOnDay|allsumDelayedByWeekDay|allsumCancelledByWeekDay| allCancelledRatio|   allDelayedRatio|allDelayed+CancelledRatio|
+---------+--------------------+----------------------+------------------------+------------------+------------------+-------------------------+
|        1|             4356643|                917639|                  119691|2.7473217337293874| 21.06298358621535|        23.81030531994474|
|        6|             3722791|                726669|                   94151| 2.529043397816316| 19.51946805501571|       22.048511452832027|
|        3|             4124239|                787556|                  109080| 2.644851571405052| 19.09578955050859|       21.740641121913644|
|        5|             4353468|                978903|                  113789|2.6137552865899094|22.485590797956938|        25.0

In [189]:
#Creates csv with specified data (DONT RUN UNLESS NEEDED)
analysisByWeekDay.write.option("header",True).csv("allanalysisByWeekDay")

22/11/24 10:20:28 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
22/11/24 10:20:28 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
22/11/24 10:20:28 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.


22/11/24 10:20:28 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
22/11/24 10:20:28 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.


22/11/24 10:20:28 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.


22/11/24 10:20:29 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.


22/11/24 10:20:30 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
